In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import pickle

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras.optimizers import Adam

In [ ]:
# !wget https://github.com/ihpar/ibu_cv/raw/main/cif_10/dataset.zip
# !unzip /content/dataset.zip -d cif_10

In [ ]:
# google colab klasör yolu
CIF_10_KLASORU = "/content/cif_10/"

# lokal klasör yolu
# CIF_10_KLASORU = "../cif_10/"

In [ ]:
def dosya_oku(dosya_yolu):
    with open(dosya_yolu, "rb") as dosya:
        return pickle.load(dosya, encoding="bytes")

In [ ]:
data_batch_1 = dosya_oku(CIF_10_KLASORU + "data_batch_1")
batches_meta = dosya_oku(CIF_10_KLASORU + "batches.meta")

print(data_batch_1.keys())
print(batches_meta.keys())

In [ ]:
tum_resimler = data_batch_1[b"data"]
print(tum_resimler.shape)

ilk_resim = tum_resimler[0]
print(ilk_resim.shape)

In [ ]:
def resmi_donustur(resim):
    resim = resim.reshape(3, 32, 32)
    resim = resim.transpose(1, 2, 0)
    return resim

def resmin_sinif_ismi(resim_index):
    sinif_sayisi = data_batch_1[b"labels"][resim_index]
    sinif_ismi = batches_meta[b"label_names"][sinif_sayisi]
    return sinif_ismi.decode("ascii")

In [ ]:
plt.figure(figsize=(8, 8))

for i in range(16):
    plt.subplot(4, 4, i+1)
    plt.title(resmin_sinif_ismi(i))
    plt.imshow(resmi_donustur(tum_resimler[i]))

plt.tight_layout()
plt.show()

### Problem 
Kedi ve gemi resimlerini ayırt edebilen bir YSA eğitmek.

### İlk aşama
Veri setini hazırlayacağız.
* Veri setinden sadece kedi ve gemi resimlerini alacağız.
* Kedilere 0; gemilere 1 etiketi takacağız.

### İkinci aşama
Çeşitli NN mimarileri oluşturup birbirleri ile karşılaştıracağız.
* Farklı katman sayıları
* Farklı nöron sayıları
* Farklı aktivasyon fonksiyonları

## Veri Seti Hazırlama

In [ ]:
tum_siniflar = np.array(data_batch_1[b"labels"])

print(tum_siniflar[:10])

In [ ]:
batches_meta[b"label_names"]


* kedinin label indeksi: 3
* gemini label indeksi: 8

In [ ]:
kediler = tum_resimler[tum_siniflar==3] / 255.0
gemiler = tum_resimler[tum_siniflar==8] / 255.0

In [ ]:
print(kediler.shape)
print(gemiler.shape)

In [ ]:
plt.figure(figsize=(8, 8))

for i in range(16):
    plt.subplot(4, 4, i+1)
    plt.imshow(resmi_donustur(kediler[i]))

plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(8, 8))

for i in range(16):
    plt.subplot(4, 4, i+1)
    plt.imshow(resmi_donustur(gemiler[i]))

plt.tight_layout()
plt.show()

In [ ]:
kedi_sayisi = kediler.shape[0]
gemi_sayisi = gemiler.shape[0]
print(kedi_sayisi, gemi_sayisi)

In [ ]:
y_kedi = np.zeros(kedi_sayisi)
y_gemi = np.ones(gemi_sayisi)
print(y_kedi.shape, y_gemi.shape)

In [ ]:
X = np.concatenate((kediler, gemiler), axis=0)
print(X.shape)

y = np.concatenate((y_kedi, y_gemi), axis=0)
print(y.shape)

In [ ]:
X_train, X_rest, y_train, y_rest = train_test_split(X, y, test_size=0.2, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_rest, y_rest, test_size=0.5, random_state=42)

In [ ]:
plt.figure(figsize=(8, 8))

for i in range(16):
    plt.subplot(4, 4, i+1)
    plt.title(str(y_train[i]))
    plt.imshow(resmi_donustur(X_train[i]))

plt.tight_layout()
plt.show()

## NN Mimarileri ve Testler

### Model 0: Tek katman, tek nöron, aktivasyon sigmoid

In [ ]:
model_0 = Sequential()
model_0.add(Dense(1, input_shape=(3072,)))
model_0.add(Activation("sigmoid"))

model_0.compile(optimizer=Adam(learning_rate=0.01), loss="binary_crossentropy", metrics=["accuracy"])
model_0.summary()

In [ ]:
history = model_0.fit(X_train, y_train, validation_data=(X_val, y_val), batch_size=32, epochs=20)

In [ ]:
print("Model 0 degerlendirmesi:")
model_0.evaluate(X_test, y_test)

In [ ]:
def plot_model_history(history): 
    plt.figure(figsize=(10, 8))
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('Model 0 Loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'validation'], loc='upper right')
    plt.show()

In [ ]:
plot_model_history(history)

In [ ]:
def plot_model_tahminleri(model, X_test, y_test):
    plt.figure(figsize=(9, 9))

    for i in range(9):
        plt.subplot(3, 3, i+1)
        prediction = model.predict([X_test[i].reshape(-1, 3072)])
        prediction = np.round(prediction[0][0], 2)
        plt.title("Gercek:" + str(y_test[i]) + ", tahmin:" + str(prediction))
        plt.imshow(resmi_donustur(X_test[i]))

    plt.tight_layout()
    plt.show()

In [ ]:
plot_model_tahminleri(model_0, X_test, y_test)

### Model 1: İlk katman 32 nöron, çıkış 1 nöron, aktivasyon sigmoid

In [ ]:
model_1 = Sequential()
model_1.add(Dense(32, activation="linear", input_shape=(3072,)))
model_1.add((Dense(1, activation="sigmoid")))

model_1.compile(optimizer=Adam(learning_rate=0.01), loss="binary_crossentropy", metrics=["accuracy"])
model_1.summary()

In [ ]:
history_1 = model_1.fit(X_train, y_train, validation_data=(X_val, y_val), batch_size=32, epochs=20)
plot_model_history(history_1)

print("Model 1 degerlendirmesi:")
model_1.evaluate(X_test, y_test)

plot_model_tahminleri(model_1, X_test, y_test)

### Model 2: İlk katman 32 nöron - aktivaston relu, çıkış 1 nöron, aktivasyon sigmoid

In [ ]:
model_2 = Sequential()
model_2.add(Dense(32, activation="relu", input_shape=(3072,)))
model_2.add((Dense(1, activation="sigmoid")))

model_2.compile(optimizer=Adam(learning_rate=0.01), loss="binary_crossentropy", metrics=["accuracy"])
model_2.summary()

In [ ]:
history_2 = model_2.fit(X_train, y_train, validation_data=(X_val, y_val), batch_size=32, epochs=20)
plot_model_history(history_2)

print("Model 2 degerlendirmesi:")
model_2.evaluate(X_test, y_test)

plot_model_tahminleri(model_2, X_test, y_test)

### Model 3: İlk katman 8 nöron - aktivaston relu, ikinci katman 4 nöron - aktivasyon relu, çıkış 1 nöron, aktivasyon sigmoid

In [ ]:
model_3 = Sequential()
model_3.add(Dense(8, activation="relu", input_shape=(3072,)))
model_3.add(Dense(4, activation="relu"))
model_3.add((Dense(1, activation="sigmoid")))

model_3.compile(optimizer=Adam(learning_rate=0.01), loss="binary_crossentropy", metrics=["accuracy"])
model_3.summary()

In [ ]:
history_3 = model_3.fit(X_train, y_train, validation_data=(X_val, y_val), batch_size=32, epochs=20)
plot_model_history(history_3)

print("Model 3 degerlendirmesi:")
model_3.evaluate(X_test, y_test)

plot_model_tahminleri(model_3, X_test, y_test)

### Model 4: İlk katman 64 nöron - aktivaston linear, ikinci katman 64 nöron - aktivasyon relu, çıkış 1 nöron, aktivasyon sigmoid, validasyon setinde iyileşme durunca eğitim erken bitecek

In [ ]:
model_4 = Sequential()
model_4.add(Dense(64, activation="linear", input_shape=(3072,)))
model_4.add(Dense(64, activation="relu"))
model_4.add((Dense(1, activation="sigmoid")))

model_4.compile(optimizer=Adam(learning_rate=0.01), loss="binary_crossentropy", metrics=["accuracy"])
model_4.summary()

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

In [ ]:
es_callback = EarlyStopping(monitor='val_loss', mode='min', patience=5, verbose=1, restore_best_weights=True)

In [ ]:
history_4 = model_4.fit(X_train, y_train, validation_data=(X_val, y_val), batch_size=16, epochs=50, callbacks=[es_callback])

plot_model_history(history_4)

print("Model 4 degerlendirmesi:")
model_4.evaluate(X_test, y_test)

plot_model_tahminleri(model_4, X_test, y_test)